<a href="https://colab.research.google.com/github/muellerzr/Practical-Deep-Learning-for-Coders-2.0/blob/master/Tabular_Numpy_DL_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install fastai2 --quiet

     |████████████████████████████████| 194kB 4.8MB/s 


In [0]:
!pip show fastai2 fastcore

Name: fastai2
Version: 0.0.17
Summary: Version 2 of the fastai library
Home-page: https://github.com/fastai/fastai2
Author: Jeremy Howard, Sylvain Gugger, and contributors
Author-email: info@fast.ai
License: Apache Software License 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: pillow, torch, pandas, scipy, spacy, matplotlib, fastprogress, pyyaml, scikit-learn, requests, torchvision, fastcore
Required-by: 
---
Name: fastcore
Version: 0.1.17
Summary: Python supercharged for fastai development
Home-page: https://github.com/fastai/fastcore
Author: Jeremy Howard and Sylvain Gugger
Author-email: infos@fast.ai
License: Apache Software License 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: dataclasses, numpy
Required-by: fastai2


In [0]:
from fastai2.tabular.all import *

In [0]:
path = untar_data(URLs.ADULT_SAMPLE)

In [0]:
df = pd.read_csv(path/'adult.csv')

In [0]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]
y_names = 'salary'

In [0]:
splits = RandomSplitter()(range_of(df))

In [0]:
to = TabularPandas(df, procs=procs, cat_names=cat_names, cont_names=cont_names,
                   y_names=y_names, splits=splits)

In [0]:
to.train.xs.iloc[:3]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
791,8,11,3,11,1,5,1,-0.555681,0.164486,2.317593
24137,5,12,3,5,1,5,1,-1.214923,-0.858567,-0.424264
22395,7,12,3,6,1,5,1,0.616307,-0.285481,-0.424264


In [0]:
to.train.ys.iloc[:3]

,salary
3661,1
1247,1
17317,0


In [0]:
dls = to.dataloaders(bs=128, device='cpu')

In [0]:
%%timeit
_ = next(iter(dls.train))

100 loops, best of 3: 19.2 ms per loop


In [0]:
%%timeit
_ = next(iter(dls.valid))

100 loops, best of 3: 3.47 ms per loop


In [0]:
%%timeit
for _ in dls.train:
    _

1 loop, best of 3: 671 ms per loop


In [0]:
%%timeit
for _ in dls.valid:
    _

10 loops, best of 3: 167 ms per loop


In [0]:
671/len(dls.train)

3.3054187192118225

In [0]:
167/len(dls.valid)

3.2745098039215685

In [0]:
dls = to.dataloaders(bs=128, device='cuda')

In [0]:
%%timeit
_ = next(iter(dls.train))

100 loops, best of 3: 19.2 ms per loop


In [0]:
%%timeit
_ = next(iter(dls.valid))

100 loops, best of 3: 3.58 ms per loop


In [0]:
%%timeit
for _ in dls.train:
    _

1 loop, best of 3: 697 ms per loop


In [0]:
%%timeit
for _ in dls.valid:
    _

10 loops, best of 3: 173 ms per loop


In [0]:
learn = tabular_learner(dls, layers=[200,100], metrics=accuracy)

In [0]:
%%time
learn.fit(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.377574,0.364423,0.833999,00:02
1,0.356772,0.357792,0.835688,00:02
2,0.358388,0.358207,0.833692,00:02
3,0.352414,0.352521,0.840602,00:02
4,0.349441,0.350070,0.840756,00:02
5,0.347263,0.358235,0.841370,00:02
6,0.346777,0.352908,0.838606,00:02
7,0.352095,0.352776,0.839681,00:02
8,0.347428,0.348187,0.840909,00:02
9,0.346684,0.352819,0.835074,00:02


CPU times: user 22.2 s, sys: 263 ms, total: 22.4 s
Wall time: 22.9 s


In [0]:
class TabDataset():
    "A `NumPy` dataset from a `TabularPandas` object"
    def __init__(self, to):
        self.cats = to.cats.to_numpy().astype(np.long)
        self.conts = to.conts.to_numpy().astype(np.float32)
        self.ys = to.ys.to_numpy()

In [0]:
ds = TabDataset(to.train)

In [0]:
ds[0]

TypeError: ignored

In [0]:
class TabDataset():
    "A `NumPy` dataset from a `TabularPandas` object"
    def __init__(self, to):
        self.cats = to.cats.to_numpy().astype(np.long)
        self.conts = to.conts.to_numpy().astype(np.float32)
        self.ys = to.ys.to_numpy()

    def __getitem__(self, idx):
        idx = idx[0]
        return self.cats[idx:idx+self.bs], self.conts[idx:idx+self.bs], self.ys[idx:idx+self.bs]

In [0]:
class TabDataset():
    "A `NumPy` dataset from a `TabularPandas` object"
    def __init__(self, to):
        self.cats = to.cats.to_numpy().astype(np.long)
        self.conts = to.conts.to_numpy().astype(np.float32)
        self.ys = to.ys.to_numpy()

    def __getitem__(self, idx):
        idx = idx[0]
        return self.cats[idx:idx+self.bs], self.conts[idx:idx+self.bs], self.ys[idx:idx+self.bs]

    def __len__(self): return len(self.cats)

In [0]:
train_ds = TabDataset(to.train)
valid_ds = TabDataset(to.valid)

In [0]:
train_ds.bs = 3

In [0]:
train_ds[[3]]

(array([[ 7, 16,  3, 13,  1,  1,  1],
        [ 8, 12,  5, 13,  2,  3,  1],
        [ 1, 12,  3,  1,  1,  5,  1]]),
 array([[-0.48341432, -1.5334858 , -0.02760405],
        [-1.509832  ,  1.8785704 , -0.41903225],
        [ 0.9828966 ,  0.509499  , -0.41903225]], dtype=float32),
 array([[0],
        [0],
        [0]], dtype=int8))

Now to build our `DataLoader`, we're going to want to modify 4 particular functions:

1. `create_item`
2. `create_batch`
3. `get_idxs`
4. `shuffle_fn`

Each of these play a particular role. First let's look at our template:

In [0]:
class TabDataLoader(DataLoader):
    def __init__(self, dataset, bs=1, num_workers=0, device='cuda', shuffle=False, **kwargs):
        "A `DataLoader` based on a `TabDataset`"
        super().__init__(dataset, bs=1, num_workers=num_workers, shuffle=shuffle, 
                         device=device, drop_last=shuffle, **kwargs)
        self.dataset.bs=bs

In [0]:
dl = TabDataLoader(train_ds, bs=3)

In [0]:
dl.dataset.bs

3

In [0]:
dl.dataset[[0]]

(array([[ 5, 10,  3,  5,  1,  5,  1],
        [ 5, 12,  3,  4,  1,  5,  1],
        [ 5,  8,  5,  2,  4,  5,  1]]),
 array([[-0.04352104,  1.6801333 ,  1.1466806 ],
        [ 0.76295   ,  3.3406117 , -0.41903225],
        [-1.0699388 ,  0.3654061 ,  0.75525236]], dtype=float32),
 array([[1],
        [1],
        [0]], dtype=int8))

In [0]:
class TabDataLoader(DataLoader):
    def __init__(self, dataset, bs=1, num_workers=0, device='cuda', shuffle=False, **kwargs):
        "A `DataLoader` based on a `TabDataset`"
        self.dataset_bs = bs
        super().__init__(dataset, bs=1, num_workers=num_workers, shuffle=shuffle, 
                         device=device, drop_last=shuffle, **kwargs)
        self.dataset.bs=self.dataset_bs
    
    def create_item(self, s): return s

    def create_batch(self, b):
        cat, cont, y = self.dataset[b]
        return tensor(cat).to(self.device), tensor(cont).to(self.device), tensor(y).to(self.device)

In [0]:
dl = TabDataLoader(train_ds, bs=128, shuffle=False)

In [0]:
b = next(iter(dl))

In [0]:
b

In [0]:
class TabDataLoader(DataLoader):
    def __init__(self, dataset, bs=1, num_workers=0, device='cuda', shuffle=False, **kwargs):
        "A `DataLoader` based on a `TabDataset`"
        super().__init__(dataset, bs=bs, num_workers=num_workers, shuffle=shuffle, 
                         device=device, drop_last=shuffle, **kwargs)
        self.dataset.bs=bs
    
    def create_item(self, s): return s

    def create_batch(self, b):
        "Create a batch of data"
        cat, cont, y = self.dataset[b]
        return tensor(cat).to(self.device), tensor(cont).to(self.device), tensor(y).to(self.device)

    def get_idxs(self):
        "Get index's to select"
        idxs = Inf.count if self.indexed else Inf.nones
        if self.n is not None: idxs = list(range(len(self.dataset)))
        return idxs

    def shuffle_fn(self):
        "Shuffle the interior dataset"
        rng = np.random.permutation(len(self.dataset))
        self.dataset.cats = self.dataset.cats[rng]
        self.dataset.conts = self.dataset.conts[rng]
        self.dataset.ys = self.dataset.ys[rng]

In [0]:
train_ds = TabDataset(to.train)
valid_ds = TabDataset(to.valid)

In [0]:
train_dl = TabDataLoader(train_ds, device='cpu', shuffle=True, bs=128)
valid_dl = TabDataLoader(valid_ds, device='cpu', bs=128)

In [0]:
o = next(iter(train_dl))

In [0]:
o[0].shape

torch.Size([128, 7])

In [0]:
len(o)

3

In [0]:
%%timeit
_ = next(iter(train_dl))

The slowest run took 4.93 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 678 µs per loop


In [0]:
%%timeit
_ = next(iter(valid_dl))

1000 loops, best of 3: 221 µs per loop


In [0]:
%%timeit
for _ in train_dl:
    _

10 loops, best of 3: 33.1 ms per loop


In [0]:
%%timeit
for _ in valid_dl:
    _

100 loops, best of 3: 8.3 ms per loop


In [0]:
train_dl = TabDataLoader(train_ds, device='cuda', shuffle=True, bs=128)
valid_dl = TabDataLoader(valid_ds, device='cuda', bs=128)

In [0]:
%%timeit
_ = next(iter(train_dl))

1000 loops, best of 3: 933 µs per loop


In [0]:
%%timeit
_ = next(iter(valid_dl))

1000 loops, best of 3: 455 µs per loop


In [0]:
%%timeit
for _ in train_dl:
    _

10 loops, best of 3: 53.2 ms per loop


In [0]:
%%timeit
for _ in valid_dl:
    _

100 loops, best of 3: 14 ms per loop


In [0]:
class TabDataLoaders(DataLoaders):
    def __init__(self, to, bs=64, val_bs=None, shuffle_train=True, device='cpu', **kwargs):
        train_ds = TabDataset(to.train)
        valid_ds = TabDataset(to.valid)
        val_bs = bs if val_bs is None else val_bs
        train = TabDataLoader(train_ds, bs=bs, shuffle=shuffle_train, device=device, **kwargs)
        valid = TabDataLoader(valid_ds, bs=val_bs, shuffle=False, device=device, **kwargs)
        super().__init__(train, valid, device=device, **kwargs)

In [0]:
dls = TabDataLoaders(to, bs=128, device='cuda')

In [0]:
emb_szs = get_emb_sz(to)
net = TabularModel(emb_szs, n_cont=3, out_sz=2, layers=[200,100]).cuda()
learn = Learner(dls, net, metrics=accuracy, loss_func=CrossEntropyLossFlat())

In [0]:
%%time
learn.fit(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.365548,0.365794,0.829392,00:01
1,0.355480,0.361641,0.833845,00:01
2,0.351890,0.358395,0.837377,00:01
3,0.348911,0.358475,0.833385,00:01
4,0.346887,0.357748,0.835995,00:01
5,0.345842,0.357555,0.836302,00:01
6,0.343665,0.357774,0.833385,00:01
7,0.343034,0.359036,0.835688,00:01
8,0.341005,0.365253,0.833538,00:01
9,0.339049,0.363166,0.831695,00:01


CPU times: user 14.5 s, sys: 174 ms, total: 14.7 s
Wall time: 15.2 s


In [0]:
(15/23) * 100

65.21739130434783

In [0]:
%%timeit
for batch in dls[1]:
    with torch.no_grad():
        learn.model.eval()
        out = learn.model(batch[0], batch[1])

10 loops, best of 3: 50.9 ms per loop


In [0]:
dls = to.dataloaders(bs=128, device='cuda')

In [0]:
%%timeit
for batch in dls[1]:
    with torch.no_grad():
        learn.model.eval()
        out = learn.model(batch[0], batch[1])

1 loop, best of 3: 210 ms per loop
